## Overview

In [1]:
import json, warnings, os
from pathlib import Path

warnings.filterwarnings('ignore')

if Path.cwd().name == "notebooks":
    rootdir = Path.cwd().parent
else:
    rootdir = Path.cwd()

import pandas as pd
import plotly.express as px
from plotly import graph_objects as go

from stables.utils.postgres import get_sqlalchemy_engine
from stables.config import local_pg_config, remote_pg_config
engine = get_sqlalchemy_engine(local_pg_config)

In [3]:
query = """
SELECT * FROM llama.token_price
"""
df= pd.read_sql(query, engine)
# df_ethena['date'] = pd.to_datetime(df_ethena['date'], unit='s')
# df[df['chain']=="Ethereum"]
df.time.value_counts()
df

,network,contract_address,symbol,decimals,confidence,price,time,_dlt_load_id,_dlt_id
0,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,0.646607,2024-04-02 13:46:59+00:00,1753194809.647163,OYo/ZNnpLd9yTw
1,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,0.960867,2024-04-03 13:46:20+00:00,1753194809.647163,VAX7ciHhos9I2A
2,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,1.095000,2024-04-04 13:46:32+00:00,1753194809.647163,l288ahrvK2HgZg
3,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,0.943694,2024-04-05 13:45:49+00:00,1753194809.647163,V2JcoQCAD/RpZQ
4,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,1.073057,2024-04-06 13:49:36+00:00,1753194809.647163,UX4zcTq+6NUU4A
...,...,...,...,...,...,...,...,...,...
472,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,0.405831,2025-07-18 13:47:48+00:00,1753194809.647163,n9eFSCE+ejffxQ
473,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,0.402120,2025-07-19 13:47:54+00:00,1753194809.647163,AIKtqvMglUS7Rg
474,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,0.488556,2025-07-20 13:47:32+00:00,1753194809.647163,N1qdXCcxpjUXdw
475,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,0.519199,2025-07-21 13:48:03+00:00,1753194809.647163,HXj+fQyQ0nhL4g


### 1. TVL

In [ ]:
df = df_ethena.copy()
l = df['date'].max()

tvl_usde = df[
    (df['date'] == l) & (df['id'] == '146')
]['circulating'].sum()

tvl_usdtb = df[
    (df['date'] == l) & (df['id'] == '221')
]['circulating'].sum()

tvl = tvl_usde + tvl_usdtb


n_chains_usde = df[
    (df['date'] == l) & (df['id'] == '146')
]['chain'].nunique()
n_chains_usdtb = df[
    (df['date'] == l) & (df['id'] == '221')
]['chain'].nunique()

In [ ]:


df = df_ethena.groupby(['date', 'symbol']).agg(
    tvl=('circulating', 'sum')
).reset_index()
fig = px.bar(df, x="date", y="tvl", color="symbol", title=f"Current TVL: {tvl/1e9:.2f}B, USDe: {tvl_usde/1e9:.2f}B, USDTB: {tvl_usdtb/1e9:.2f}B",labels="")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    legend_title_text="Stablecoin"
)
fig.show()

### 2. TVL by chains

In [ ]:
df = df_ethena.copy()
df = df.groupby(['date', 'chain']).agg(
    tvl=('circulating', 'sum')
).reset_index()
fig = px.bar(df, x="date", y="tvl", color="chain", title="Ethena TVL by chains")
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    legend_title_text="Chain"
)
fig.show()

### 3. USDe staking percentage

In [ ]:
df1 = df_ethena[df_ethena['id'] == '146'].groupby(['date']).agg(tvl=('circulating', 'sum')).reset_index()

query = """
SELECT * FROM ethena_raw.susde_pool
"""
df2 = pd.read_sql(query, engine)
df2['date'] = df2['timestamp'].dt.date
df2['date'] = pd.to_datetime(df2['date'])

In [ ]:
df = df1.merge(df2[['date', 'tvl_usd']], on='date', how='outer')#, suffixes=('_usde', '_sus)
df.rename(columns={'tvl_usd': 'susde', 'tvl': 'usde'}, inplace=True)
df['staking_perc'] = (df['susde'] / df['usde']) *100
current_staking_perc = df[df['date'] == df['date'].max()]['staking_perc'].values[0]

In [ ]:
fig = go.Figure()
# fig.add_trace(go.Scatter(x=df2['date'], y=df2['tvl_usd'], mode='lines', name='sUSDe'))
fig.add_trace(go.Bar(x=df1['date'], y=df1['tvl'], name='USDe', opacity=0.5))

fig.add_trace(go.Scatter(
      x=df['date'],
      y=df['staking_perc'],
      mode='lines',
      name='Staking %',
      yaxis='y2'
  ))
fig.update_layout(
      yaxis2=dict(
          title="%",
          overlaying='y',
          side='right'
      ),
    title="Current USDe TVL: {:.2f}B, staking {:.2f}%".format(tvl_usde/1e9, current_staking_perc),
    xaxis_title="Date",
    yaxis_title="TVL (USD)",
    # legend_title_text="Stablecoin"
  )


### 4. Revenue

In [2]:
query = """
SELECT * FROM ethena_raw.protocol_revenue_ethena
"""
df = pd.read_sql(query, engine)
# df['date'] = df['timestamp'].dt.date
df['date'] = pd.to_datetime(df['timestamp'], unit='s').dt.date
df['revenue_cumulative'] = df['revenue'].cumsum()
df

,timestamp,revenue,protocol,_dlt_load_id,_dlt_id,chain,sub_protocol,date,revenue_cumulative
0,1700870400,0,ethena,1753107066.1861103,hq9hVFxJP6szDQ,Ethereum,Ethena USDe,2023-11-25,0
1,1700956800,0,ethena,1753107066.1861103,6CqQJnPnRQPk8A,Ethereum,Ethena USDe,2023-11-26,0
2,1701043200,0,ethena,1753107066.1861103,Y8kah/Y+XdqnCQ,Ethereum,Ethena USDe,2023-11-27,0
3,1701129600,103,ethena,1753107066.1861103,1Sl2GrcPSfPbgw,Ethereum,Ethena USDe,2023-11-28,103
4,1701216000,3,ethena,1753107066.1861103,Oi0EnSGyfgghdw,Ethereum,Ethena USDe,2023-11-29,106
...,...,...,...,...,...,...,...,...,...
594,1752624000,1661,ethena,1753107066.1861103,gRonI5cPkRlIYQ,Ethereum,Ethena USDe,2025-07-16,402701110
595,1752710400,7601364,ethena,1753107066.1861103,ck5+U7QV4YFz0Q,Ethereum,Ethena USDe,2025-07-17,410302474
596,1752796800,145676,ethena,1753107066.1861103,JVS2kxYOIg1s/w,Ethereum,Ethena USDe,2025-07-18,410448150
597,1752883200,165763,ethena,1753107066.1861103,wXu7PtLRGLokng,Ethereum,Ethena USDe,2025-07-19,410613913


In [3]:
px.line(df, x="date", y="revenue_cumulative", color="protocol")

### 4. ENA
1. ENA total supply

In [ ]:
query = """
SELECT * FROM ethena_raw.token_price
"""
df = pd.read_sql(query, engine)
df['date'] = pd.to_datetime(df['timestamp'], unit='s').dt.date
df

,timestamp,price,network,contract_address,symbol,decimals,confidence,_dlt_load_id,_dlt_id,date
0,1712109083,0.857356,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,1753149022.9225566,RgtV7Z8XdHhmbg,2024-04-03
1,1712195509,1.099000,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,1753149022.9225566,bz2rU+RTbH3kAw,2024-04-04
2,1712281881,1.022000,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,1753149022.9225566,/5jjw5MM6LvH0w,2024-04-05
3,1712368185,0.978615,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,1753149022.9225566,84DpK4Hcnpa3LQ,2024-04-06
4,1712454628,1.081625,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,1753149022.9225566,OXAfJgBcnNDWew,2024-04-07
...,...,...,...,...,...,...,...,...,...,...
471,1752803396,0.394288,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,1753149022.9225566,5lXlOSd0MTLa5A,2025-07-18
472,1752889803,0.390960,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,1753149022.9225566,reUCJjPFIWUtSA,2025-07-19
473,1752976172,0.445386,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,1753149022.9225566,mk7IYVSoUw+x7g,2025-07-20
474,1753062596,0.494682,ethereum,0x57e114B691Db790C35207b2e685D4A43181e6061,ENA,18,0.99,1753149022.9225566,G+xOk80U6MyPDQ,2025-07-21


In [8]:
ena_total_supply = 15e9
current_price = df[df['date'] == df['date'].max()]['price'].values[0].item()  # Ensure it's a scalar value
current_price

full_market_cap = ena_total_supply * current_price

print(f"ENA Total Supply: {ena_total_supply}")
print(f"Current Price: {current_price}")
print(f"Full Market Cap: {full_market_cap/1e9:.2f}B")

ENA Total Supply: 15000000000.0
Current Price: 0.511373
Full Market Cap: 7.67B


## USDe

### 1. Primary market